In [1]:
%matplotlib inline
#from keras.applications.resnet50 import ResNet50
# not using keras version since it cannot have an input size < 197x197.  Might also require 3-channel color.
import keras
from keras.optimizers import RMSprop
from keras.utils import np_utils
import resnet
from glob import glob
import numpy as np
import os
import random
from PIL import Image
import matplotlib.pyplot as plt
import json

Using TensorFlow backend.
/home/rallen/anaconda3/envs/dl/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#K.image_data_format()
print(keras.backend.image_data_format())
print(keras.backend.image_dim_ordering())

channels_last
tf


In [3]:
train_path = 'data/train3'
train_input_image_names = glob(train_path+'/crop_score*png')
train_input_image_names.sort()
train_input_images = np.stack([np.array(Image.open(fn)) for fn in train_input_image_names])
train_input_images.shape = train_input_images.shape + (1,)
num_train_images,rows,cols,channels = train_input_images.shape
num_train_images,rows,cols,channels

(10078, 120, 32, 1)

In [4]:
valid_path = 'data/valid3'
valid_input_image_names = glob(valid_path+'/crop_score*png')
valid_input_image_names.sort()
valid_input_images = np.stack([np.array(Image.open(fn)) for fn in valid_input_image_names])
valid_input_images.shape = valid_input_images.shape + (1,)
num_valid_images,rows,cols,channels = valid_input_images.shape
num_valid_images,rows,cols,channels

(3359, 120, 32, 1)

In [5]:
results_json = 'data/train3/crop_score_info.json'
with open(results_json,'r') as f:
    results = json.load(f)

In [6]:
train_indices = []
for train_name in train_input_image_names:
    index = int(train_name.split('.')[0].split('_')[-1])
    train_indices.append(index)
valid_indices = []
for valid_name in valid_input_image_names:
    index = int(valid_name.split('.')[0].split('_')[-1])
    valid_indices.append(index)
#train_indices[:20],valid_indices[:20]

In [7]:
notes = np.zeros(128+1,dtype=np.int32)
lengths = np.zeros(16+1,dtype=np.int32)
for ti in train_indices:
    r = results[ti] # [84, 4] = 
    notes[r[0]] += 1
    lengths[r[1]] += 1

In [8]:
# HMM, realizing that MULTIPLE classes of output (note + length) are not 
# easily mapped here...going to need to look at this harder.  For now
# let's just try to see if we can make due with 2 different models.
#np.concatenate([lengths,notes])
#NUM_CATEGORIES=128+1 + 16+1
NUM_CATEGORIES=128+1 
NUM_CATEGORIES1=16+1
#train_results = np.zeros([len(train_indices),NUM_CATEGORIES],dtype=np.float32)
train_results = np.zeros(len(train_indices),dtype=np.float32)
train_results1 = np.zeros(len(train_indices),dtype=np.float32)
i = 0
for ti in train_indices:
    r = results[ti]
    #train_results[i,16+1+r[0]] = 1.0 # set note value
    #train_results[i,r[1]] = 1.0 # set duration value
    train_results[i] = r[0]
    train_results1[i] = r[1]
    i += 1
train_results = np_utils.to_categorical(train_results,NUM_CATEGORIES)
train_results1 = np_utils.to_categorical(train_results1,NUM_CATEGORIES1)

In [9]:
train_results.shape, train_results1.shape

((10078, 129), (10078, 17))

In [10]:
#valid_results = np.zeros([len(valid_indices),128+1 + 16+1],dtype=np.float32)
valid_results = np.zeros(len(valid_indices),dtype=np.float32)
valid_results1 = np.zeros(len(valid_indices),dtype=np.float32)
i = 0
for ti in valid_indices:
    r = results[ti]
    #valid_results[i,16+1+r[0]] = 1.0 # set note value
    #valid_results[i,r[1]] = 1.0 # set duration value
    valid_results[i] = r[0]
    valid_results1[i] = r[1]
    i += 1
valid_results = np_utils.to_categorical(valid_results,NUM_CATEGORIES)
valid_results1 = np_utils.to_categorical(valid_results1,NUM_CATEGORIES1)

In [11]:
train_results[100], train_results1[100]

(array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.]))

In [12]:
class score_generator(object):
    """X should be list of score images, Y should be a list of score results.  
    Both should be the same length."""
    def __init__(self, X, Y, batch_size, channels):
        self.X  = X
        self.Y  = Y
        self.bs = batch_size
        self.channels = channels
        self.i  = 0
    def __next__(self):
        xs = self.X[self.i:self.i+self.bs]
        ys = self.Y[self.i:self.i+self.bs]
        #ys = ys.reshape(len(ys),-1,1)
        self.i = (self.i + self.bs) % self.X.shape[0]
        return xs, ys

In [13]:
batch_size = 32
train_generator = score_generator(train_input_images, train_results, batch_size, channels)
valid_generator = score_generator(valid_input_images, valid_results, batch_size, channels)
train_generator1 = score_generator(train_input_images, train_results1, batch_size, channels)
valid_generator1 = score_generator(valid_input_images, valid_results1, batch_size, channels)

In [14]:
#model = resnet.ResnetBuilder.build_resnet_18((64, 64, 1), 128) # Total params: 11,444,288
#model = resnet.ResnetBuilder.build_resnet_34((64, 64, 1), 128) # Total params: 21,563,584
#model = resnet.ResnetBuilder.build_resnet_50((64, 64, 1), 128) # Total params: 24,025,920
image_shape = (rows,cols,channels)
model = resnet.ResnetBuilder.build_resnet_18(image_shape, NUM_CATEGORIES)
model1 = resnet.ResnetBuilder.build_resnet_18(image_shape, NUM_CATEGORIES1)

In [15]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 120, 32, 1)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 60, 16, 64)   3200        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 60, 16, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 60, 16, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [16]:
train_rate  = 1e-4
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(train_rate),
              metrics=["accuracy"])
model1.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(train_rate),
              metrics=["accuracy"])

In [17]:
num_epochs = 25
model.fit_generator(train_generator,
                    num_train_images//batch_size, num_epochs,
                    verbose=2,
                    validation_data=valid_generator,
                    validation_steps=num_valid_images//batch_size)

Epoch 1/25
 - 15s - loss: 1.7527 - acc: 0.7923 - val_loss: 0.9854 - val_acc: 0.8705
Epoch 2/25
 - 13s - loss: 0.8229 - acc: 0.9266 - val_loss: 0.7455 - val_acc: 0.9387
Epoch 3/25
 - 13s - loss: 0.6393 - acc: 0.9813 - val_loss: 0.6121 - val_acc: 0.9747
Epoch 4/25
 - 13s - loss: 0.5655 - acc: 0.9931 - val_loss: 0.5642 - val_acc: 0.9832
Epoch 5/25
 - 13s - loss: 0.5192 - acc: 0.9934 - val_loss: 0.5349 - val_acc: 0.9825
Epoch 6/25
 - 14s - loss: 0.4730 - acc: 0.9945 - val_loss: 0.4910 - val_acc: 0.9816
Epoch 7/25
 - 13s - loss: 0.4300 - acc: 0.9944 - val_loss: 0.4762 - val_acc: 0.9789
Epoch 8/25
 - 13s - loss: 0.3980 - acc: 0.9967 - val_loss: 0.4080 - val_acc: 0.9864
Epoch 9/25
 - 14s - loss: 0.3633 - acc: 0.9975 - val_loss: 0.3868 - val_acc: 0.9852
Epoch 10/25
 - 13s - loss: 0.3395 - acc: 0.9971 - val_loss: 0.3824 - val_acc: 0.9819
Epoch 11/25
 - 13s - loss: 0.3163 - acc: 0.9975 - val_loss: 0.3382 - val_acc: 0.9891
Epoch 12/25
 - 13s - loss: 0.2816 - acc: 0.9989 - val_loss: 0.3078 - val_a

In [18]:
model1.fit_generator(train_generator1,
                    num_train_images//batch_size, num_epochs,
                    verbose=2,
                    validation_data=valid_generator1,
                    validation_steps=num_valid_images//batch_size)

Epoch 1/25
 - 15s - loss: 1.3490 - acc: 0.8010 - val_loss: 1.0109 - val_acc: 0.8618
Epoch 2/25
 - 13s - loss: 0.7506 - acc: 0.9329 - val_loss: 0.8610 - val_acc: 0.9179
Epoch 3/25
 - 13s - loss: 0.6167 - acc: 0.9748 - val_loss: 0.7947 - val_acc: 0.9402
Epoch 4/25
 - 13s - loss: 0.5532 - acc: 0.9886 - val_loss: 0.7503 - val_acc: 0.9465
Epoch 5/25
 - 13s - loss: 0.5114 - acc: 0.9922 - val_loss: 0.7092 - val_acc: 0.9546
Epoch 6/25
 - 13s - loss: 0.4716 - acc: 0.9939 - val_loss: 0.7304 - val_acc: 0.9467
Epoch 7/25
 - 13s - loss: 0.4350 - acc: 0.9967 - val_loss: 0.6732 - val_acc: 0.9566
Epoch 8/25
 - 13s - loss: 0.4080 - acc: 0.9969 - val_loss: 0.5452 - val_acc: 0.9714
Epoch 9/25
 - 13s - loss: 0.3830 - acc: 0.9964 - val_loss: 0.5833 - val_acc: 0.9590
Epoch 10/25
 - 13s - loss: 0.3599 - acc: 0.9974 - val_loss: 0.5658 - val_acc: 0.9626
Epoch 11/25
 - 13s - loss: 0.3281 - acc: 0.9973 - val_loss: 0.5423 - val_acc: 0.9653
Epoch 12/25
 - 13s - loss: 0.3035 - acc: 0.9981 - val_loss: 0.5182 - val_a

In [19]:
test_path = 'data/test3'
test_input_image_names = glob(test_path+'/crop_score*png')
test_input_image_names.sort()
test_input_images = np.stack([np.array(Image.open(fn)) for fn in test_input_image_names])
test_input_images.shape = test_input_images.shape + (1,)
num_test_images,rows,cols,channels = test_input_images.shape
num_test_images,rows,cols,channels

(3359, 120, 32, 1)

In [20]:
test_indices = []
for test_name in test_input_image_names:
    index = int(test_name.split('.')[0].split('_')[-1])
    test_indices.append(index)

In [21]:
test_results = np.zeros(len(test_indices),dtype=np.float32)
test_results1 = np.zeros(len(test_indices),dtype=np.float32)
i = 0
for ti in test_indices:
    r = results[ti]
    test_results[i] = r[0]
    test_results1[i] = r[1]
    i += 1
test_results = np_utils.to_categorical(test_results,NUM_CATEGORIES)
test_results1 = np_utils.to_categorical(test_results1,NUM_CATEGORIES1)

In [22]:
test_generator = score_generator(test_input_images, test_results, batch_size, channels)
test_generator1 = score_generator(test_input_images, test_results1, batch_size, channels)

In [23]:
model.evaluate_generator(test_generator,steps=num_test_images//batch_size)

[0.16556710133758876, 0.99038461538461542]

In [24]:
model1.evaluate_generator(test_generator1,steps=num_test_images//batch_size)

[0.32224668462115985, 0.96875]

In [25]:
# not bad!  Definitely overfitting going on, but to be expected with not a huge amount of training data.
# prev. comment was before 4x data.  Now getting great correlation after just 25 iters.